<div class="alert alert-block alert-danger">
TODO change to the  code from lectures and change  progression:
    <ol> 
    <li>  They put the Iris data into the visualisations they made last week but this time using the class labels (provbide rough code ion case they didn;t do last week but encourage thme to use their own)</li>
        <li> MCQ's to confirm that one class is seperable, the others are not </li>
    <li>  Take my 1-NN code from the lectures, and the k-NN pseudocode and implement kNN for iris</li>
    <li> Sanity check: create a nre wclassifier using the sklearn class and verify their results are in the same ball park (not statistically significantly different?)  </li>
<li> Wrap-up discussion about comparing algorithms and when it is safe to say methodX is better than method Y. Realte back to disc ussinos in new second week workbook, topical real world - relate to comparison of vaccines, or different treatments for covid
    </ol></div>

# Workbook 6: Supervised Machine Learning

## Description and aims

This tutorial is designed to give you your first experience of machine learning in practice by implementing a simple nearest-neighbour classifier.

The learning outcomes are:
- experience of implementing a classification algorithm
- experience of working with a data set, such as:   putting it into a suitable format; visualisation,  checking the test and train set are representative
-  experience of working through different preprocessing steps to try and improve the performance of your classifier

## Activity 1: Getting to know your data: Group/Individual 10 minutes
## CHANGE FROM PLOTLY TO WORKBOOK 5 CODE
Splitting data into train and test

The attached files contain a dataset in a choice of formats: excel or tab-delimited text.

There are 150 examples, each having values for 4 different features.

The first steps are to:
split the data into a training and a test set:   I suggest 100 training and 50 test.
You might find it easiest to do this in excel by cut-pasting a set of rows into a different region of the workbook
or you can just save them in to different text files.
Use visualisation tools to make sure that both the sets are similar in terms of:
The proportion of different classes in each
The spread of values for each feature.

How to visualise your data:

We'll use an online tool called plotly to do this.
1. Go to https://plot.ly/create/#/
   you should see an area for data in the top right
2. Copy and paste your training set data into the tab named 'unnamed grid 1' - just the 4 columns x 100 rows of numerical data,  not the class labels
3. click on the +sign to create another tab in the data box - it should create an empty table called unnamed grid 2
4.  Copy and paste your  test  set data into the tab named 'unnamed grid 2' -  just  the 4 columns x 50 rows of numerical  data,  not the  class  labels

5. Now in the middle pane click the +trace button to add a new trace showing the distribution of values in the training data.  
  Click where it says 'scatter'. and change the type to 'violin'
  Click on the X box and select 'unnamed grid 1 A'.  leave the Y box blank
  This should fill the plot box below with a 'violin' plot that shows the rough distribution of values for feature A of your training set.  The x values are the actual values and the height at different points of the shape shows how frequent those values were.

6. Repeat step 5 but this time use the new trace to plot 'unnamed grid 2 A' - the first column of your test data
  you should see the two violin plots on the same figure,  so you can compare by eye to make sure they look similar.

7. Repeat steps 5-6 for the different features a b c and d 

## Activity 2: Class Discussion 5 minutes

We will have a quick class discussion to share any thoughts about the nature of the data that came up from activity 1.
Things to think about:
How could we make sure our training and test data sets are both representative?
How can we make sure that all our variables take equal importance when we measure distance between two points?
What would we have done if we had missing values for variables in some of the data items??

<div class="alert alert-block alert-danger"> Please save your work (click the save icon) then shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook</div>

<div class="alert alert-block alert-danger"> Remember to download and save your work if you are not running this notebook locally.</div>## Activity 3: Individual- Coding a k-Nearest Neighbour Classifier in python


The main activity for this session is to implement 1-NN in Python and use the training and test sets you have created to test your implementation.

There's a two-minute video explaining the algorithm here:  https://www.geeksforgeeks.org/k-nearest-neighbours/

Note that like most videos they show a dataset where each example is described by values for just two variables (features).

We want code that will work in the more general case where we tell it how many features for reach individual dataset.
 
We also want you to write your code in the form of aq class with a cosntructor, and methods fit() and predict() so it is in the same form as all the modelling classes from sklearn.

To save you time the pseudocode for predict using 1-NN is reproduced below.

NOTEBOX: 


###Predict function 
// Measure distance of new point to existing examples
// N is the number of iutems in your training set  
array distances[N] 
For (i=0 TO N) 
    Set distances[i] = CalculateDistance(newSample, dataset[i]) 

// Get the one with the smallest value 
set closest = 0 
for (i=0 to N) 
    if (distances[i] < distances[closest] 
        set closest = i 

// Return the label of the closest point as a prediction 
RETURN datasetlabels[closest] 


###CalculateDistance (Sample1, Sample2) 
// This example does Euclidean (straight line) distance 
set distance = 0 
FOR (feature = 0 TO feature = NUM_FEATURES -1) 
    set difference = Sample1[feature] – Sample2[feature] 
    set distance = distance + difference * difference 
set distance = SquareRoot(distance) 
RETURN Distance



<div class="alert alert-block alert-danger"> Please save your work (click the save icon) then shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook</div>

<div class="alert alert-block alert-danger"> Remember to download and save your work if you are not running this notebook locally.</div>